<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/_Deep_Learning_for_Quantum_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# Generator Network
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(128),
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            nn.Linear(256, output_dim),  # Output dimension should be 28*28 for MNIST-like images
            nn.Tanh()  # Output in range [-1, 1]
        )
        # Initialize weights
        for m in self.fc:
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.fc(x)

# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),  # Output a single scalar
        )
        # Initialize weights
        for m in self.fc:
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.fc(x)  # No sigmoid here, BCEWithLogitsLoss expects raw logits

# Initialize Models and Optimizers
z_dim = 100  # Latent dimension
data_dim = 28 * 28  # Flattened image dimension for MNIST (28x28 = 784)
generator = Generator(z_dim, data_dim)
discriminator = Discriminator(data_dim)

optim_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optim_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Learning rate scheduler (optional)
scheduler_g = optim.lr_scheduler.StepLR(optim_g, step_size=30, gamma=0.1)
scheduler_d = optim.lr_scheduler.StepLR(optim_d, step_size=30, gamma=0.1)

# Loss function (using BCEWithLogitsLoss for stability)
criterion = nn.BCEWithLogitsLoss()

# MNIST Dataset loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Helper function to save generator outputs
def save_generator_output(epoch, generator, z_dim, num_images=5):
    noise = torch.randn(num_images, z_dim)  # Generate random latent vectors
    generated_images = generator(noise).detach().numpy()
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    for i in range(num_images):
        axes[i].imshow(generated_images[i].reshape(28, 28), cmap='gray')
        axes[i].axis('off')
    plt.suptitle(f"Epoch {epoch}")
    plt.savefig(f'generated_images_epoch_{epoch}.png')  # Save the images

# Training Loop
epochs = 100

# Tracking loss
d_losses = []
g_losses = []

for epoch in range(epochs):
    for real_batch, _ in train_loader:
        real_batch = real_batch.view(-1, data_dim)  # Flatten MNIST images
        real_batch_size = real_batch.size(0)  # Get the actual batch size

        # Generate fake data with the same batch size
        noise = torch.randn(real_batch_size, z_dim)  # Match batch size
        fake_data = generator(noise).detach()

        # Train Discriminator
        optim_d.zero_grad()
        real_labels = torch.ones(real_batch_size, 1) * 0.9  # Label smoothing: real data labels (0.9 instead of 1)
        fake_labels = torch.zeros(real_batch_size, 1)  # Fake data labels (0)

        # Calculate loss for real and fake data
        loss_d_real = criterion(discriminator(real_batch), real_labels)
        loss_d_fake = criterion(discriminator(fake_data), fake_labels)
        loss_d = (loss_d_real + loss_d_fake) / 2  # Average the losses
        loss_d.backward()
        optim_d.step()

        # Train Generator
        optim_g.zero_grad()
        # The generator wants to fool the discriminator into thinking its fake data is real
        loss_g = criterion(discriminator(generator(noise)), real_labels)  # We want fake data to be labeled as real
        loss_g.backward()
        optim_g.step()

    # Scheduler step (optional)
    scheduler_g.step()
    scheduler_d.step()

    # Track losses
    d_losses.append(loss_d.item())
    g_losses.append(loss_g.item())

    # Print and visualize periodically
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss D: {loss_d.item():.4f}, Loss G: {loss_g.item():.4f}")
        save_generator_output(epoch, generator, z_dim)  # Save generated images

    # Save the model periodically
    if epoch % 50 == 0:
        torch.save(generator.state_dict(), f'generator_epoch_{epoch}.pth')
        torch.save(discriminator.state_dict(), f'discriminator_epoch_{epoch}.pth')

# Plot the loss curves
plt.plot(d_losses, label="Discriminator Loss")
plt.plot(g_losses, label="Generator Loss")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Losses")
plt.savefig("training_losses.png")

# Final model save
torch.save(generator.state_dict(), 'generator_final.pth')
torch.save(discriminator.state_dict(), 'discriminator_final.pth')